In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


## Data Preprocessing

In [36]:
train_df = pd.read_csv(r'C:\Users\susha\Downloads\archive (8)\FraudDetection\fraudTrain.csv', index_col="Unnamed: 0")
test_df = pd.read_csv(r'C:\Users\susha\Downloads\archive (8)\FraudDetection\fraudTest.csv',  index_col="Unnamed: 0")

In [22]:
test_df.columns


Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [23]:
train_df.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [24]:
train_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.171565e+17,7.027910e+01,4.880159e+04,3.853336e+01,-9.022626e+01,8.905776e+04,1.344906e+09,3.853346e+01,-9.022648e+01,5.727773e-03
std,1.308811e+18,1.599518e+02,2.689804e+04,5.076852e+00,1.375858e+01,3.024351e+05,1.019700e+07,5.111233e+00,1.377093e+01,7.546503e-02
min,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,1.800400e+14,9.640000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.336682e+09,3.472954e+01,-9.689864e+01,0.000000e+00
50%,3.520550e+15,4.745000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.344902e+09,3.936295e+01,-8.743923e+01,0.000000e+00
75%,4.642260e+15,8.305000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.354366e+09,4.195602e+01,-8.023228e+01,0.000000e+00
max,4.992350e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.362932e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1048575 non-null  object 
 1   cc_num                 1048575 non-null  float64
 2   merchant               1048575 non-null  object 
 3   category               1048575 non-null  object 
 4   amt                    1048575 non-null  float64
 5   first                  1048575 non-null  object 
 6   last                   1048575 non-null  object 
 7   gender                 1048575 non-null  object 
 8   street                 1048575 non-null  object 
 9   city                   1048575 non-null  object 
 10  state                  1048575 non-null  object 
 11  zip                    1048575 non-null  int64  
 12  lat                    1048575 non-null  float64
 13  long                   1048575 non-null  float64
 14  city_pop          

In [26]:
fraud_count = train_df['is_fraud'].value_counts()
fraud_count

0    1042569
1       6006
Name: is_fraud, dtype: int64

In [27]:
fraud_count = test_df['is_fraud'].value_counts()
fraud_count

0    553574
1      2145
Name: is_fraud, dtype: int64

In [28]:
print(train_df.isna().sum().sum())


0


In [29]:
print(train_df.duplicated().sum())


0


#### There are way too many unnecessary attributes in the dataset, so we'll remove all of them at first before visualizing 

In [37]:
train_df.drop(columns=['cc_num','first', 'last','zip', 'dob', 'trans_num','merch_lat', 'merch_long', 'lat', 'long'],inplace=True)


In [38]:
train_df

,trans_date_trans_time,merchant,category,amt,gender,street,city,state,city_pop,job,unix_time,is_fraud
0,1/1/2019 0:00,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,561 Perry Cove,Moravian Falls,NC,3495,"Psychologist, counselling",1325376018,0
1,1/1/2019 0:00,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,43039 Riley Greens Suite 393,Orient,WA,149,Special educational needs teacher,1325376044,0
2,1/1/2019 0:00,fraud_Lind-Buckridge,entertainment,220.11,M,594 White Dale Suite 530,Malad City,ID,4154,Nature conservation officer,1325376051,0
3,1/1/2019 0:01,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,9443 Cynthia Court Apt. 038,Boulder,MT,1939,Patent attorney,1325376076,0
4,1/1/2019 0:03,fraud_Keeling-Crist,misc_pos,41.96,M,408 Bradley Rest,Doe Hill,VA,99,Dance movement psychotherapist,1325376186,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,3/10/2020 16:07,fraud_Fadel Inc,health_fitness,77.00,F,05561 Farrell Crescent,Annapolis,MD,92106,"Accountant, chartered certified",1362931649,0
1048571,3/10/2020 16:07,"fraud_Cremin, Hamill and Reichel",misc_pos,116.94,F,043 Hanson Turnpike,Hedrick,IA,1583,Geochemist,1362931670,0
1048572,3/10/2020 16:08,"fraud_O'Connell, Botsford and Hand",home,21.27,F,005 Cody Estates,Louisville,KY,736284,Engineering geologist,1362931711,0
1048573,3/10/2020 16:08,fraud_Thompson-Gleason,health_fitness,9.52,F,576 House Crossroad,West Sayville,NY,4056,Film/video editor,1362931718,0
